In [311]:
import pandas as pd
from collections import Counter 
from helper import load_data,pipeline, unique_drugs, generate_drugs_df
import re

%pylab inline 
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [312]:
trials = load_data()
c = unique_drugs(trials)
df_data, exceptions = generate_drugs_df(c, trials)
trials.reset_index(inplace=True)


## expanded unique drugs to get a dataframe we can group by
dupes = []
for idx, item_list in enumerate(trials["Primary Drugs"]):
    split_list = item_list.split(",")
    if len(split_list)>1:
        for i in split_list:
            trials.append(trials.iloc[idx,:])
            trials.iloc[-1,-3]=i
        dupes.append(idx)



got 512 results and had 0 exceptions
probs are[22.66, 29.31, 67.65]


In [313]:
trials.shape

(1870, 44)

In [314]:
split_drug_names = trials.drop(dupes)

drug_list = split_drug_names["Primary Drugs"].unique()
phase_of_trial_list = split_drug_names["Phase of Trial"].unique()
df_num_of_pariticpants = pd.DataFrame(np.zeros([len(drug_list),len(phase_of_trial_list)]),columns=phase_of_trial_list)
df_num_of_pariticpants['drug'] = drug_list
df_num_of_pariticpants = df_num_of_pariticpants.set_index('drug')
grouped_phases = split_drug_names.groupby(["Primary Drugs","Phase of Trial"])["Planned Subject Number"].agg('sum')
cleaner_groups = grouped_phases.reset_index()

for drug in drug_list:
    for item in cleaner_groups[cleaner_groups["Primary Drugs"]==drug].iterrows():   
        phase = list(item)[1]['Phase of Trial']
        count = list(item)[1]['Planned Subject Number']
        df_num_of_pariticpants.loc[drug,phase] = count

    
    

In [315]:
len(dupes)

416

In [317]:
df_data.merge(df_num_of_pariticpants, left_on=["drug"], right_on=["drug"], how='inner').shape

(431, 21)

In [286]:
df_data

,drug,num_of_trials,phase_1_success,phase_2_success,phase_3_success,phase_1_2_success,phase_2_3_success,phase_1_start,phase_1_first_end,phase_1_last_end,phase_2_start,phase_2_first_end,phase_2_last_end
5,Trastuzumab,184,True,True,True,True,False,2002-04-01,2002-05-01,2017-06-20,1998-04-01,2001-01-01,2016-03-11
22,Lapatinib,111,True,True,True,True,False,2003-04-28,2003-11-01,2012-09-01,2002-10-01,2003-12-09,2014-05-01
21,Paclitaxel,98,True,True,True,True,True,2005-12-01,2006-02-10,2015-01-12,1998-04-01,1999-11-01,2014-08-01
23,Bevacizumab,94,True,True,True,True,False,2005-09-01,2005-10-01,2014-12-09,2001-11-01,2001-12-01,2013-11-01
9,Capecitabine,80,True,True,True,True,True,2000-08-01,2001-12-01,2013-09-01,1998-07-01,2001-09-01,2016-01-23
24,Docetaxel,75,True,True,True,True,True,1996-12-01,1997-01-01,2011-01-01,1997-01-01,1997-06-01,2012-09-01
31,Letrozole,55,True,True,True,True,False,2010-11-01,2011-08-09,2010-11-01,2000-01-01,2004-05-01,2014-11-01
16,Anastrozole,54,True,True,True,True,False,2012-04-01,2012-08-01,2017-08-14,2000-10-01,2003-01-01,2014-11-01
27,Eribulin,51,True,True,True,True,False,2011-10-01,2011-11-01,2013-06-01,2004-09-01,2006-09-20,2015-10-28
10,Everolimus,50,True,True,True,True,False,2005-11-01,2008-03-01,2012-01-01,2005-01-19,2005-08-03,2014-11-01
